# Transfer learning by using the VTID2 dataset

---

### Importing the libraries

In [8]:
import tensorflow as tf
from tensorflow.keras import layers, utils
from tensorflow import keras
from matplotlib import pyplot as plt

import sys
sys.path.insert(0, '../helpers/')
from plot_graphs import plot_loss, plot_acc

### Setting batch and image size, and importing the datasets 

#### Main dataset - Zenodo

In [9]:
IMAGE_SIZE = 100
BATCH_SIZE = 32

train_zenodo = utils.image_dataset_from_directory("../datasets/data/split/Zenodo/train", image_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE)
val_zenodo = utils.image_dataset_from_directory("../datasets/data/split/Zenodo/val", image_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE)
test_zenodo = utils.image_dataset_from_directory("../datasets/data/split/Zenodo/test", image_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE)

train_zenodo = train_zenodo.prefetch(buffer_size=tf.data.AUTOTUNE).cache()
val_zenodo = val_zenodo.prefetch(buffer_size=tf.data.AUTOTUNE).cache()
test_zenodo = test_zenodo.prefetch(buffer_size=tf.data.AUTOTUNE).cache()

Found 3840 files belonging to 6 classes.
Found 480 files belonging to 6 classes.
Found 480 files belonging to 6 classes.


#### Secondary dataset - VTID2

In [10]:
train_vtid2 = utils.image_dataset_from_directory("../datasets/data/split/VTID2/train", image_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE)
val_vtid2 = utils.image_dataset_from_directory("../datasets/data/split/VTID2/val", image_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE)
test_vtid2 = utils.image_dataset_from_directory("../datasets/data/split/VTID2/test", image_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE)

train_vtid2 = train_vtid2.prefetch(buffer_size=tf.data.AUTOTUNE).cache()
val_vtid2 = val_vtid2.prefetch(buffer_size=tf.data.AUTOTUNE).cache()
test_vtid2 = test_vtid2.prefetch(buffer_size=tf.data.AUTOTUNE).cache()

Found 3484 files belonging to 5 classes.
Found 435 files belonging to 5 classes.
Found 437 files belonging to 5 classes.


## 1. Creating a model for the VTID2 dataset

> This is the model architecture for the best model from notebook 3_tuning_model_parameters with all default hyperparameters, but ran for the VTID2 dataset which has five classes. 

In [15]:
inputs = keras.Input(shape=(100, 100, 3))
x = inputs
x = layers.Rescaling(1./255)(x)
x = layers.Conv2D(32, 3, activation='relu')(x)
x = layers.Conv2D(32, 3, activation='relu')(x)
x = layers.Conv2D(32, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)
x = layers.Conv2D(64, 3, activation='relu')(x)
x = layers.Conv2D(64, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)
x = layers.Conv2D(128, 3, activation='relu')(x)
x = layers.Conv2D(128, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)
x = layers.Conv2D(256, 3, activation='relu')(x)
x = layers.Conv2D(256, 3, activation='relu')(x)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(5, activation='softmax')(x)

model_5_vtid2 = keras.Model(inputs, outputs)

model_5_vtid2.compile(optimizer=keras.optimizers.RMSprop(0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model_5_vtid2.summary()

callbacks_list = [
    keras.callbacks.EarlyStopping(
        monitor="val_accuracy",
        patience=10
    ),
    keras.callbacks.ModelCheckpoint(
        filepath="../models/5_vtid2.h5",
        monitor="val_accuracy",
        save_best_only=True
    )
]

history_5_vtid2 = model_5_vtid2.fit(
  train_vtid2,
  epochs=40,
  callbacks=callbacks_list,
  validation_data=val_vtid2
)

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 100, 100, 3)]     0         
                                                                 
 rescaling_2 (Rescaling)     (None, 100, 100, 3)       0         
                                                                 
 conv2d_18 (Conv2D)          (None, 98, 98, 32)        896       
                                                                 
 conv2d_19 (Conv2D)          (None, 96, 96, 32)        9248      
                                                                 
 conv2d_20 (Conv2D)          (None, 94, 94, 32)        9248      
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 47, 47, 32)       0         
 2D)                                                             
                                                           

2022-08-29 09:38:30.245316: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


109/109 [==============================] - ETA: 0s - loss: 1.5797 - accuracy: 0.3106

2022-08-29 09:38:40.856616: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


109/109 [==============================] - 12s 97ms/step - loss: 1.5797 - accuracy: 0.3106 - val_loss: 1.4992 - val_accuracy: 0.3931
Epoch 2/40
109/109 [==============================] - 10s 94ms/step - loss: 1.3129 - accuracy: 0.4673 - val_loss: 1.5658 - val_accuracy: 0.4138
Epoch 3/40
109/109 [==============================] - 10s 93ms/step - loss: 0.9703 - accuracy: 0.6404 - val_loss: 0.7945 - val_accuracy: 0.6828
Epoch 4/40
109/109 [==============================] - 10s 93ms/step - loss: 0.7350 - accuracy: 0.7331 - val_loss: 0.6720 - val_accuracy: 0.7333
Epoch 5/40
109/109 [==============================] - 10s 94ms/step - loss: 0.5476 - accuracy: 0.8048 - val_loss: 0.4802 - val_accuracy: 0.8299
Epoch 6/40
109/109 [==============================] - 10s 96ms/step - loss: 0.4018 - accuracy: 0.8639 - val_loss: 0.4089 - val_accuracy: 0.8621
Epoch 7/40
109/109 [==============================] - 11s 97ms/step - loss: 0.2831 - accuracy: 0.9087 - val_loss: 0.3653 - val_accuracy: 0.8897
Epo

In [23]:
load_vtid2 = keras.models.load_model("../models/5_vtid2.h5")
load_vtid2.evaluate(test_vtid2)

2022-08-29 09:59:53.551749: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


14/14 [==============================] - 1s 35ms/step - loss: 0.2059 - accuracy: 0.9817


[0.2059335559606552, 0.9816934466362]

> This model worked extremely well for the VTID2 dataset with a test accuracy of 98.17%.

## 2. Partially trained model

> Importing the trained VTID2 model shown in the section above, and training it from the 4th Conv2D layer onwards for the Zenodo dataset.

In [24]:
vtid2 = keras.models.load_model('../models/5_vtid2.h5')
vtid2.layers
vtid2.trainable = False

for layer in vtid2.layers[6:]:
  layer.trainable = True

x = vtid2.layers[-2].output
outputs = layers.Dense(6, activation='softmax', name='classifier')(x)
model_partially_trained = keras.Model(vtid2.inputs, outputs)
model_partially_trained.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model_partially_trained.summary()

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 100, 100, 3)]     0         
                                                                 
 rescaling_2 (Rescaling)     (None, 100, 100, 3)       0         
                                                                 
 conv2d_18 (Conv2D)          (None, 98, 98, 32)        896       
                                                                 
 conv2d_19 (Conv2D)          (None, 96, 96, 32)        9248      
                                                                 
 conv2d_20 (Conv2D)          (None, 94, 94, 32)        9248      
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 47, 47, 32)       0         
 2D)                                                             
                                                           

In [25]:
callbacks_list = [
    keras.callbacks.EarlyStopping(
        monitor="val_accuracy",
        patience=10
    ),
    keras.callbacks.ModelCheckpoint(
        filepath="../models/5_partially_trainable.h5",
        monitor="val_accuracy",
        save_best_only=True
    )
]

history_partially_trained = model_partially_trained.fit(
  train_zenodo,
  epochs=50,
  callbacks=callbacks_list,
  validation_data=val_zenodo
)

Epoch 1/50


2022-08-29 10:01:14.609316: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


120/120 [==============================] - ETA: 0s - loss: 1.0543 - accuracy: 0.6063

2022-08-29 10:01:22.463792: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


120/120 [==============================] - 9s 64ms/step - loss: 1.0543 - accuracy: 0.6063 - val_loss: 0.5408 - val_accuracy: 0.8146
Epoch 2/50
120/120 [==============================] - 7s 60ms/step - loss: 0.4615 - accuracy: 0.8383 - val_loss: 0.3878 - val_accuracy: 0.8604
Epoch 3/50
120/120 [==============================] - 7s 60ms/step - loss: 0.2782 - accuracy: 0.9042 - val_loss: 0.3359 - val_accuracy: 0.8813
Epoch 4/50
120/120 [==============================] - 7s 60ms/step - loss: 0.1747 - accuracy: 0.9391 - val_loss: 0.3709 - val_accuracy: 0.9021
Epoch 5/50
120/120 [==============================] - 7s 61ms/step - loss: 0.1524 - accuracy: 0.9549 - val_loss: 0.4003 - val_accuracy: 0.8875
Epoch 6/50
120/120 [==============================] - 7s 60ms/step - loss: 0.0951 - accuracy: 0.9690 - val_loss: 1.9205 - val_accuracy: 0.7729
Epoch 7/50
120/120 [==============================] - 7s 61ms/step - loss: 0.0977 - accuracy: 0.9763 - val_loss: 0.4025 - val_accuracy: 0.9167
Epoch 8/50

In [26]:
load_partially_trainable = keras.models.load_model("../models/5_partially_trainable.h5")
load_partially_trainable.evaluate(test_zenodo)

2022-08-29 10:07:22.997842: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


15/15 [==============================] - 1s 31ms/step - loss: 0.5899 - accuracy: 0.9146


[0.5899118781089783, 0.9145833849906921]

> This gives a test accuracy of 91.46% which definitely does not beat the best model for the Zenodo dataset. 

## 3. All Trainable

> In this model, we again use the trained VTID2 model from section 1 of this notebook, but this time, set all layer to trainable and train it on the Zenodo dataset.

In [28]:
vtid2 = keras.models.load_model('../models/5_vtid2.h5')

x = vtid2.layers[-2].output
outputs = keras.layers.Dense(6, activation='softmax', name='classifier')(x)
all_trainable = keras.Model(vtid2.inputs, outputs)
all_trainable.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
all_trainable.summary()

Model: "model_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 100, 100, 3)]     0         
                                                                 
 rescaling_2 (Rescaling)     (None, 100, 100, 3)       0         
                                                                 
 conv2d_18 (Conv2D)          (None, 98, 98, 32)        896       
                                                                 
 conv2d_19 (Conv2D)          (None, 96, 96, 32)        9248      
                                                                 
 conv2d_20 (Conv2D)          (None, 94, 94, 32)        9248      
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 47, 47, 32)       0         
 2D)                                                             
                                                           

In [29]:
callbacks_list = [
    keras.callbacks.EarlyStopping(
        monitor="val_accuracy",
        patience=10
    ),
    keras.callbacks.ModelCheckpoint(
        filepath="../models/5_all_trainable.h5",
        monitor="val_accuracy",
        save_best_only=True
    )
]

history_all_trainable = all_trainable.fit(
  train_zenodo,
  epochs=50,
  callbacks=callbacks_list,
  validation_data=val_zenodo
)

Epoch 1/50


2022-08-29 10:11:45.516599: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


120/120 [==============================] - ETA: 0s - loss: 1.0694 - accuracy: 0.6031

2022-08-29 10:11:56.982958: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


120/120 [==============================] - 12s 93ms/step - loss: 1.0694 - accuracy: 0.6031 - val_loss: 0.5586 - val_accuracy: 0.8021
Epoch 2/50
120/120 [==============================] - 11s 91ms/step - loss: 0.4844 - accuracy: 0.8276 - val_loss: 0.4009 - val_accuracy: 0.8458
Epoch 3/50
120/120 [==============================] - 11s 91ms/step - loss: 0.2913 - accuracy: 0.9068 - val_loss: 0.2875 - val_accuracy: 0.8833
Epoch 4/50
120/120 [==============================] - 11s 90ms/step - loss: 0.1954 - accuracy: 0.9331 - val_loss: 1.2331 - val_accuracy: 0.7500
Epoch 5/50
120/120 [==============================] - 11s 91ms/step - loss: 0.1555 - accuracy: 0.9518 - val_loss: 0.3250 - val_accuracy: 0.9021
Epoch 6/50
120/120 [==============================] - 11s 92ms/step - loss: 0.1772 - accuracy: 0.9560 - val_loss: 0.2584 - val_accuracy: 0.9083
Epoch 7/50
120/120 [==============================] - 11s 92ms/step - loss: 0.1017 - accuracy: 0.9669 - val_loss: 0.5204 - val_accuracy: 0.8708
Epo

In [32]:
all_trainable_model = keras.models.load_model("../models/5_all_trainable.h5")
all_trainable_model.evaluate(test_zenodo)

 1/15 [=>............................] - ETA: 2s - loss: 0.5552 - accuracy: 0.9688

2022-08-29 10:24:56.086883: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


15/15 [==============================] - 1s 31ms/step - loss: 0.5740 - accuracy: 0.9438


[0.5739955306053162, 0.9437500238418579]

> This model gives a test accuracy of 94.38% which is excellent, and comes very close to the best accuracy of Zenodo model. But the fact remains that despite using transfer learning, it does not beat the best accuracy of the best Zenodo model.